In [20]:
import numpy as np
import matplotlib.pyplot as plt
import math
import ipympl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,HBox, Layout,VBox
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import DataStructs
from rdkit.Chem import AllChem
import nglview as nv
from nglview.viewer_control import ViewerControl
import mplcursors
import os
from urllib.parse import parse_qs,urlparse, quote
import IPython.display
from IPython.core.display import HTML

In [17]:
from tools.load_molecule_data import load_data_file
from tools.helper_functions import calc_scaling,rotate_molecule
from tools.calc_average import full_average_IR, full_average_R_pol, full_average_conv_pol
from tools.calc_single import single_rot_IR,single_rot_R
from tools.plotting_functions import create_average_spec_single
from tools.load_database import get_target

In [18]:
try:
    query = os.environ.get('QUERY_STRING', '')
    parameters = parse_qs(query)
    molcode=parameters.get('Molcode')[0]
    SH=int(parameters.get('SH')[0])
except:
    molcode="000-823-384"
    SH=0

if SH:
    molfilename="data_SH/"+molcode+".mol"  
    filename="data_SH/"+molcode+".dat"   
else:
    molfilename="data_SAu/"+molcode+".mol"  
    filename="data_SAu/"+molcode+".dat"   

In [23]:
server = os.environ.get('SERVER_NAME', 'localhost') 
url = "http://" + server

port = os.environ.get('SERVER_PORT', '')
if port:
    url += ":" + port
path = os.environ.get('SCRIPT_NAME', '')
if path:
    url += path
else:
    url="http://localhost:8888/voila/render/Documents/Work/Work_website/molecular-vibration-explorer/molecule.ipynb"
url="/".join(url.split("/")[:-1])

smiles_in="N#CCc1c(S)cc(cc1Br)Br"
find_url=url+ "/search.ipynb?smiles="+quote(smiles_in)

index_link = widgets.HTML(
    value="<a href="+ url+ "/index.ipynb>Go back to Index</a>",
    layout=Layout(height='35px'),
)

database_link = widgets.HTML(
    value="<a href="+ url+ "/database.ipynb?SH={}".format(SH)+">Go back to database</a>",
    layout=Layout(height='35px'),
)

modes_link = widgets.HTML(
    value="<a href="+ url+ "/modes.ipynb" +\"?Molcode="+molcode+"&SH={}".format(SH)+">Check normal modes</a>",
    #layout=Layout(height='10px'),
)

Find_text = widgets.Text(
    placeholder='e.g. N#CCc1c(S)cc(cc1Br)Br',
    disabled=False, 
    layout=Layout(width='90%',height='36px')
)

Find_button = widgets.HTML(
    value=f"<a href="+ find_url+ "><font size=5>Go</a>",
    layout=Layout(width='45px',height='30px',border='solid',margin='0 0 0 0'),
)


def Find_callback(wdgt):
    find_url=url+ "/search.ipynb?smiles="+quote(wdgt.value)
    Find_button.value=f"<a href="+ find_url+ "><font size=5>Go</a>"

Find_text.on_submit(Find_callback)


Find_panel=VBox([widgets.Label(value='Enter SMILES code and press $\mathbf{Enter}$:'),
                 Find_text, Find_button],
               layout=Layout(width='40%')
               )
Link_panel=VBox([index_link,database_link,modes_link],layout=Layout(width='90%'))
display(HBox([Link_panel,Find_panel]))

In [5]:
fr,Q,D,P,nat,aniso=load_data_file(filename)

# read in coordinates from mol file for displaying vibrations
def read_mol(molfilename):
    molcoords=[]
    atoms=[]
    with open(molfilename,'r') as inpfile:
        line=inpfile.readline()
        while line:
            spl=line.split()
            if len(spl)==16:
                molcoords.append([float(spl[0]),float(spl[1]),float(spl[2])])
                atoms.append(spl[3])
            line=inpfile.readline()
    return molcoords,atoms

molcoords,atoms=read_mol(molfilename)

with open(molfilename, 'r') as file:
    data = file.readlines()
                    

smiles=data[2]
mol=Chem.MolFromSmiles(smiles)


In [6]:
def get_target_single(intens,target_type="P"):    
    if target_type=="P": # conversion
        Tmean=-28.367672289689146 
        Tstd=0.3809242121851108
    elif target_type=="R": # Raman Stokes
        Tmean=-28.91883480924085
        Tstd=0.2625275063027046  
    else: # IR absorption
        Tmean=2.290063042859388
        Tstd=0.22266072753602675
    sint=np.sum(intens)
    target_=np.log10(sint, out=np.full_like(sint,np.NINF),where=sint!=0)    
    target=(target_-Tmean)/Tstd
    return target

def update_orientation(phi,theta,xi):
    th=theta*torad  
    ph=phi*torad
    x=xi*torad
    
    rotaxes,cell=rotate_molecule(atoms,molcoords,molfilename,phi=ph,theta=th,xi=x)
    
    control.spin([1,0,0],th)
    control.spin([0,0,1],ph)
    
    mol_view.shape.add('text', list(8*rotaxes[0]+[0, 0, 0.5]), [ 0, 0, 1 ], 3, 'x')
    mol_view.shape.add('text', list(8*rotaxes[1]+[0, 0, 0.5]), [ 1, 0, 0 ], 3, 'y')
    mol_view.shape.add('text', list(8*rotaxes[2]+[0, 0.5, 0]), [ 0, 0.8, 0.2 ], 3, 'z')
    mol_view.shape.add_arrow([0,0,0], list(8*rotaxes[0]), [ 0, 0, 1 ], 0.2, 'x')
    mol_view.shape.add_arrow([0,0,0], list(8*rotaxes[1]), [ 1, 0, 0 ], 0.2, 'y')
    mol_view.shape.add_arrow([0,0,0],  list(8*rotaxes[2]), [ 0, 0.8, 0.2 ], 0.2, 'z')

def plot_spectrum_oriented(laser,T,freqrange,sclf,broadening,gammaIR,gammaR,phi,theta,xi,show_av,show_single,
                           rb_ir_beam,rb_rin_beam,rb_rout_beam):
    # modes: list of normal modes to calculate
    modes=range(0,len(fr)) 

    torad=math.pi/180
    th=theta*torad  
    ph=phi*torad 
    x=xi*torad 
    Lm=1
    
    # calculate intensity scaling factors
    v0= math.pow(10, 7)/laser
    scalingIR,scaling,scalingexp,scalingpolar= calc_scaling(T)

    # Full orientation averages
    nummodes=len(modes)
    conv_av=np.zeros((nummodes))  # conversion intensity
    ir_av=np.zeros((nummodes))    # IR intensity
    r_av=np.zeros((nummodes))     # Raman Stokes intensity, parallel fields

    # Single orientations
    ir_single=np.zeros((nummodes))   # IR intensity
    r_single=np.zeros((nummodes))    # Raman Stokes, parallel fields
    conv_single=np.zeros((nummodes)) # Conversion intensity, parallel fields

    maxdiff=0
    maxdiffP=0
    for n,m in enumerate(modes):

    # Calculate frequency-dependent scaling factors
        # Usual Stokes for thermal population
        scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
                    fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
        # Usual anti-Stokes for thermal population
        scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
               1/(-1+math.exp(-scalingexp * fr[m]))) # 
        # For THOR: anti-Stokes without thermal population
        scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] 

    # Set up field polarization vectors  
        polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
        e=polarizations[rb_ir_beam]
        e_in=polarizations[rb_rin_beam]
        e_out=polarizations[rb_rout_beam]        
        
        if show_av:
            # Calculate intensities for full orientation average
            conv_av[n] =scalingIR *scalingTHOR*full_average_conv_pol(D[m],P[m],e,e_in,e_out)
            ir_av[n]=scalingIR*full_average_IR(D[m]) 
            r_av[n]=scalingR*full_average_R_pol(P[m],e_in,e_out) 
            
        if show_single:
           # Calculate intensities for a single orientation
            ir=single_rot_IR(D[m],e,a=ph,b=th,c=x) 
            r=single_rot_R(P[m],e_in,e_out,a=ph,b=th,c=x)
            ir_single[n]=scalingIR*ir
            r_single[n]=scalingR*r
            conv_single[n]=scalingIR*scalingTHOR*ir*r

    xmin=freqrange[0]
    xmax=freqrange[1]
    res=0.2                      
    
    wn,R_spec,IR_spec,conv_spec,freqs,prod_ints,R_ints,IR_ints=create_average_spec_single(fr,ir_single, 
                                                                        r_single,conv_single,xmin,xmax,
                                                                        res,gammaIR,gammaR,sclf)
    wn,R_spec_av,IR_spec_av,conv_spec_av,freqs,prod_ints_av,R_ints_av,IR_ints_av=create_average_spec_single(
                                                                        fr,ir_av,r_av,conv_av,xmin,xmax,
                                                                        res,gammaIR,gammaR,sclf)    
    
    
    pmin=0 
    pmax=int((xmax-xmin)/res)
    maxpr=1
    maxI=1
    maxR=1
    maxpr0=0
    maxI0=0
    maxR0=0
    if show_single:
        maxpr0=np.max(prod_ints)
        maxI0=np.max(IR_ints)
        maxR0=np.max(R_ints)
        if broadening=='broadened':
            ax3.fill_between(wn,R_spec,alpha=0.6,color='b',label='Raman')
            line1=ax3.plot(wn,R_spec,alpha=0.6,color='b',label='Raman')
            ax2.fill_between(wn,IR_spec,alpha=0.6,color='r',label='THz/IR') 
            line2=ax2.plot(wn,IR_spec,alpha=0.6,color='r',label='THz/IR')
            ax1.fill_between(wn,conv_spec,alpha=0.6,color='purple',label='Conversion') 
            line3=ax1.plot(wn,conv_spec,alpha=0.6,color='purple',label='Conversion')
            maxpr0=np.max(conv_spec[pmin:pmax]) 
            maxI0=np.max(IR_spec[pmin:pmax])
            maxR0=np.max(R_spec[pmin:pmax])
            c1 = mplcursors.cursor(line1)
            c2 = mplcursors.cursor(line2)
            c3 = mplcursors.cursor(line3)

        elif broadening=='broadened+stick':
            ax3.fill_between(wn,(gammaR*math.pi)/2*R_spec,alpha=0.6,color='b',label='Raman')
            ax2.fill_between(wn,(gammaIR*math.pi)/2*IR_spec,alpha=0.6,color='r',label='THz/IR') 
            ax1.fill_between(wn,(gammaIR*math.pi)/2*(gammaR*math.pi)/2*conv_spec,alpha=0.6,color='purple',label='Conversion') 

            markerline, stemline, baseline, =ax2.stem(freqs,IR_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='THz/IR')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            c4 = mplcursors.cursor(markerline)
            markerline, stemline, baseline, =ax3.stem(freqs,R_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Raman')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            c5 = mplcursors.cursor(markerline)
            markerline, stemline, baseline, =ax1.stem(freqs,prod_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Conversion')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)
            c6 = mplcursors.cursor(markerline)

        elif broadening=='stick':
            markerline, stemline, baseline, =ax2.stem(freqs,IR_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='THz/IR')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)
            c1 = mplcursors.cursor(markerline)
            markerline, stemline, baseline, =ax3.stem(freqs,R_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Raman')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            c2 = mplcursors.cursor(markerline)
            markerline, stemline, baseline, =ax1.stem(freqs,prod_ints,
                                                     'k',basefmt='k', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Conversion')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)
            c3 = mplcursors.cursor(markerline)
        maxpr=maxpr0
        maxI=maxI0
        maxR=maxR0
    if show_av:
        maxpr1=np.max(prod_ints_av) 
        maxI1=np.max(IR_ints_av) 
        maxR1=np.max(R_ints_av) 
        if broadening=='broadened':
            ax3.fill_between(wn,R_spec_av,alpha=0.6,color='grey',label='Raman av')
            ax2.fill_between(wn,IR_spec_av,alpha=0.6,color='grey',label='THz/IR av')
            ax1.fill_between(wn,conv_spec_av,alpha=0.6,color='grey',label='Conv. av')
            maxpr1=np.max(conv_spec_av[pmin:pmax]) 
            maxI1=np.max(IR_spec_av[pmin:pmax])
            maxR1=np.max(R_spec_av[pmin:pmax])
           
        elif broadening=='broadened+stick':
            ax3.fill_between(wn,(gammaR*math.pi)/2*R_spec_av,alpha=0.6,color='grey',label='Raman av')
            ax2.fill_between(wn,(gammaIR*math.pi)/2*IR_spec_av,alpha=0.6,color='grey',label='THz/IR av')
            ax1.fill_between(wn,(gammaIR*math.pi)/2*(gammaR*math.pi)/2*conv_spec_av,alpha=0.6,color='grey',label='Conv. av')

            markerline, stemline, baseline, =ax2.stem(freqs,IR_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='THz/IR')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            
            markerline, stemline, baseline, =ax3.stem(freqs,R_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Raman')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            markerline, stemline, baseline, =ax1.stem(freqs,prod_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Conversion')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            
        elif broadening=='stick':
            markerline, stemline, baseline, =ax2.stem(freqs,IR_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='THz/IR')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            markerline, stemline, baseline, =ax3.stem(freqs,R_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Raman')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
            markerline, stemline, baseline, =ax1.stem(freqs,prod_ints_av,
                                                     'grey',basefmt='grey', markerfmt='ok',
                                                      use_line_collection=True,bottom=0,label='Conversion')
            plt.setp(stemline, linewidth = 1.25,alpha=0.6)
            plt.setp(markerline, markersize = 2,alpha=0.6)  
    
        maxpr=max(maxpr0, maxpr1) 
        maxI=max(maxI0,maxI1)
        maxR=max(maxR0,maxR1) 
    plt.xlim(xmin,xmax)
    ax1.set_ylim(-maxpr/100,1.2*maxpr)
    ax2.set_ylim(-maxI/100,1.2*maxI)
    ax3.set_ylim(-maxR/100,1.2*maxR)
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.setp(ax3.get_xticklabels(), visible=False)
    ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
    ax2.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
    ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
  #  ax2.set_title(title)
    plt.xlabel('Wavenumber /cm$^{-1}$')
    ax2.set_ylabel('THz/IR',color='r')
    ax3.set_ylabel('Raman',color='b')
    ax1.set_ylabel('Conversion',color='purple')
    ax1.yaxis.set_label_coords(-0.07,0.5)
    ax2.yaxis.set_label_coords(-0.07,0.5)
    ax3.yaxis.set_label_coords(-0.07,0.5)
        
def update_targets(laser,T,freqrange,sclf,gammaIR,gammaR,phi,theta,xi,show_av,show_single,
                    rb_ir_beam,rb_rin_beam,rb_rout_beam):
    modes=range(0,len(fr))
    torad=math.pi/180
    th=theta*torad  
    ph=phi*torad
    x=xi*torad
    Lm=1
    
    # calculate intensity scaling factors
    v0= math.pow(10, 7)/laser
    scalingIR,scaling,scalingexp,scalingpolar= calc_scaling(T)

    # Full orientation averages
    nummodes=len(modes)
    conv_av=np.zeros((nummodes))  # conversion intensity
    ir_av=np.zeros((nummodes))    # IR intensity
    r_av=np.zeros((nummodes))     # Raman Stokes intensity, parallel fields

    # Single orientations
    ir_single=np.zeros((nummodes))   # IR intensity
    r_single=np.zeros((nummodes))    # Raman Stokes, parallel fields
    conv_single=np.zeros((nummodes)) # Conversion intensity, parallel fields

    maxdiff=0
    maxdiffP=0
    for n,m in enumerate(modes):
        # Calculate frequency-dependent scaling factors
        # Usual Stokes for thermal population
        scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
                    fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
        # Usual anti-Stokes for thermal population
        scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
               1/(-1+math.exp(-scalingexp * fr[m]))) # 
        # For THOR: anti-Stokes without thermal population
        scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] 

        # Set up field polarization vectors  
        polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
        e=polarizations[rb_ir_beam]
        e_in=polarizations[rb_rin_beam]
        e_out=polarizations[rb_rout_beam]        
        
        if show_av:
            # Calculate intensities for full orientation average
            conv_av[n] =scalingIR *scalingTHOR*full_average_conv_pol(D[m],P[m],e,e_in,e_out)
            ir_av[n]=scalingIR*full_average_IR(D[m]) 
            r_av[n]=scalingR*full_average_R_pol(P[m],e_in,e_out) 
            
        if show_single:
            # Calculate intensities for a single orientation
            ir=single_rot_IR(D[m],e,a=ph,b=th,c=x) 
            r=single_rot_R(P[m],e_in,e_out,a=ph,b=th,c=x)
            ir_single[n]=scalingIR*ir
            r_single[n]=scalingR*r
            conv_single[n]=scalingIR*scalingTHOR*ir*r

    xmin=freqrange[0]
    xmax=freqrange[1]
    res=0.2                      
    
    wn,R_spec,IR_spec,conv_spec,freqs,prod_ints,R_ints,IR_ints=create_average_spec_single(fr,ir_single, 
                                                                        r_single,conv_single,xmin,xmax,
                                                                        res,gammaIR,gammaR,sclf)
    wn,R_spec_av,IR_spec_av,conv_spec_av,freqs,prod_ints_av,R_ints_av,IR_ints_av=create_average_spec_single(
                                                                        fr,ir_av,r_av,conv_av,xmin,xmax,
                                                                        res,gammaIR,gammaR,sclf)
    textA.value="{:.2f}".format(get_target_single(IR_ints,"A"))
    textR.value="{:.2f}".format(get_target_single(R_ints,"R"))
    textP.value="{:.2f}".format(get_target_single(prod_ints,"P"))
    textAav.value="{:.2f}".format(get_target_single(IR_ints_av,"A"))
    textRav.value="{:.2f}".format(get_target_single(R_ints_av,"R"))
    textPav.value="{:.2f}".format(get_target_single(prod_ints_av,"P"))
        
        
def update_box(theta, phi, xi):
    th=theta*torad  
    ph=phi*torad
    x=xi*torad
    
    rotaxes,cell=rotate_molecule(atoms,molcoords,molfilename,phi=ph,theta=th,xi=x)
    textgap.value="{:.2f}".format(cell[2])
    textsurf.value="{:.2f}".format(cell[0]*cell[1])
    

def on_value_change(change):
    ax1.clear()
    ax2.clear()
    ax3.clear()
    plot_spectrum_oriented(laser_widget.value,T_widget.value,freq_widget.value,sclf_widget.value,
                           broadening_widget.value,gammaIR_widget.value,gammaR_widget.value,phi_widget.value,
                           theta_widget.value,xi_widget.value,av_widget.value,single_widget.value,
                           rb_ir_beam.value,rb_rin_beam.value,rb_rout_beam.value) 
    update_targets(laser_widget.value,T_widget.value,freq_widget.value,sclf_widget.value,
                    gammaIR_widget.value,gammaR_widget.value,phi_widget.value,
                    theta_widget.value,xi_widget.value,av_widget.value,single_widget.value,
                    rb_ir_beam.value,rb_rin_beam.value,rb_rout_beam.value)
    
    
def on_rot_change(change):
    ax1.clear()
    ax2.clear()
    ax3.clear()
    plot_spectrum_oriented(laser_widget.value,T_widget.value,freq_widget.value,sclf_widget.value,
                           broadening_widget.value,gammaIR_widget.value,gammaR_widget.value,phi_widget.value,
                           theta_widget.value,xi_widget.value,av_widget.value,single_widget.value,
                           rb_ir_beam.value,rb_rin_beam.value,rb_rout_beam.value) 
    update_box(theta_widget.value,phi_widget.value,xi_widget.value)
    update_targets(laser_widget.value,T_widget.value,freq_widget.value,sclf_widget.value,
                    gammaIR_widget.value,gammaR_widget.value,phi_widget.value,
                    theta_widget.value,xi_widget.value,av_widget.value,single_widget.value,
                    rb_ir_beam.value,rb_rin_beam.value,rb_rout_beam.value)
    nb_c=mol_view.n_components
    nb_static=1
    for rep in range(nb_c-nb_static):
        mol_view.clear_representations(rep+nb_static)
    update_orientation(phi_widget.value,theta_widget.value,xi_widget.value)

### Spectroscopic properties

In [7]:
%matplotlib widget
from ipywidgets  import interactive_output


textlayout=widgets.Layout(width='100px')
labellayout=widgets.Layout(width='180px')
layout=widgets.Layout(width='280px', description_width='220px')
style = {'description_width': '130px'}
targetlayout=widgets.Layout(width='100px')
targetlabellayout=widgets.Layout(width='40px')


laser_widget=widgets.BoundedFloatText(value=785,min=500,max=1100.0,step=0.1,description='Raman laser /nm:',
                                      layout=layout,
                                        style=style)#{'description_width': 'initial'})
T_widget=widgets.BoundedFloatText(value=298.15,min=200,max=500.0,step=0.01,description='Temperature /K:',
                                  layout=layout,
                                    style=style)
freq_label=widgets.Label(value='Frequency range /cm-1:',layout = widgets.Layout(display="flex", justify_content="flex-end",width='150px'))
freq_widget=widgets.FloatRangeSlider(min=0,max=3900,value=[600,1700],step=1,
                    orientation='horizontal',description='', #'Frequency range /cm$^{-1}$:', 
                    layout=layout,
                    style=style,continuous_update=False)

sclf_widget=widgets.FloatSlider(value=0.98,min=0.9,max=1.1,step=0.01,description='Frequency scaling:',
                                layout=layout,
                                  style=style)
broadening_widget=widgets.RadioButtons(options=[('broadened'), ('stick'),('broadened+stick')],
                                       layout=layout,
                                         description='Type of spectrum:',style=style)
gammaIR_widget=widgets.FloatSlider(value=10, description='THz/IR FWHM /cm-1:', max=50, min=0.1,layout=layout,style=style,continuous_update=False)
gammaR_widget=widgets.FloatSlider(value=10, description='Raman FWHM /cm-1:', max=50, min=0.1,layout=layout,style=style,continuous_update=False)
phi_widget=widgets.BoundedFloatText(value=0,min=0,max=360.0,step=0.1,description=r'$\phi$ (rotate around z'+"'):",
                                    layout=layout,
                                      style=style)
theta_widget=widgets.BoundedFloatText(value=0,min=0,max=90.0,step=0.1,description=r'$\theta$ (rotate around x):',
                                      layout=layout,
                                        style=style)
xi_widget=widgets.BoundedFloatText(value=0,min=0,max=360.0,step=0.1,description=r'$\xi$ (rotate around z):',
                                    layout=layout,
                                      style=style)
single_widget=widgets.Checkbox(value=True,description='Show selected orientation',layout=layout,indent=False)
av_widget=widgets.Checkbox(value=False,description='Show full orientation average',layout=layout,indent=False)


axes=[["x",0], ["y",1] ,["z",2]]
layout_rb=widgets.Layout(height='70px',width='260px')
rb_ir_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description='THz/IR beam:',
                        layout=layout_rb,
                        style=style
                    )
rb_rin_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{in}$ beam:',
                        layout=layout_rb,
                        style=style
                    )
rb_rout_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{out}$ beam:',
                        layout=layout_rb,
                        style=style
                    )



# initiate calculations 
modes=range(0,len(fr))
torad=math.pi/180
th=theta_widget.value*torad  
ph=phi_widget.value*torad
x=xi_widget.value*torad
Lm=1

rotaxes,cell=rotate_molecule(atoms,molcoords,molfilename,phi=ph,theta=th,xi=x) # rotaxes not used

# calculate intensity scaling factors
v0= math.pow(10, 7)/laser_widget.value
scalingIR,scaling,scalingexp,scalingpolar= calc_scaling(T_widget.value)

# Full orientation averages
nummodes=len(modes)
conv_av=np.zeros((nummodes))  # conversion intensity
ir_av=np.zeros((nummodes))    # IR intensity
r_av=np.zeros((nummodes))     # Raman Stokes intensity, parallel fields

# Single orientations
ir_single=np.zeros((nummodes))   # IR intensity
r_single=np.zeros((nummodes))    # Raman Stokes, parallel fields
conv_single=np.zeros((nummodes)) # Conversion intensity, parallel fields

maxdiff=0
maxdiffP=0
for n,m in enumerate(modes):
    # Calculate frequency-dependent scaling factors
    # Usual Stokes for thermal population
    scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
            fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
    # Usual anti-Stokes for thermal population
    scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
            1/(-1+math.exp(-scalingexp * fr[m]))) # 
    # For THOR: anti-Stokes without thermal population
    scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] 

    # Set up field polarization vectors  
    polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
    e=polarizations[rb_ir_beam.value]
    e_in=polarizations[rb_rin_beam.value]
    e_out=polarizations[rb_rout_beam.value]        
        
    if av_widget.value:
        # Calculate intensities for full orientation average
        conv_av[n] =scalingIR *scalingTHOR*full_average_conv_pol(D[m],P[m],e,e_in,e_out)
        ir_av[n]=scalingIR*full_average_IR(D[m]) 
        r_av[n]=scalingR*full_average_R_pol(P[m],e_in,e_out) 
            
    if single_widget.value:
        # Calculate intensities for a single orientation
        ir=single_rot_IR(D[m],e,a=ph,b=th,c=x) 
        r=single_rot_R(P[m],e_in,e_out,a=ph,b=th,c=x)
        ir_single[n]=scalingIR*ir
        r_single[n]=scalingR*r
        conv_single[n]=scalingIR*scalingTHOR*ir*r

xmin=freq_widget.value[0]
xmax=freq_widget.value[1]
res=0.2                      
    
wn,R_spec,IR_spec,conv_spec,freqs,prod_ints,R_ints,IR_ints=create_average_spec_single(fr,ir_single, 
                                                                    r_single,conv_single,xmin,xmax,res,
                                                                    gammaIR_widget.value,gammaR_widget.value,sclf_widget.value)
wn,R_spec_av,IR_spec_av,conv_spec_av,freqs,prod_ints_av,R_ints_av,IR_ints_av=create_average_spec_single(
                                                                    fr,ir_av,r_av,conv_av,xmin,xmax,res,
                                                                    gammaIR_widget.value,gammaR_widget.value,sclf_widget.value) 



# display molecule in 3D
mol_view = nv.NGLWidget()
mol_view.control.zoom(0.3)
comp = mol_view.add_component(molfilename) 
control = ViewerControl(view=mol_view)
control.spin([1,0,0],th)
control.spin([0,0,1],ph)
    
# show axes
mol_view.shape.add('text', list(8*rotaxes[0]+[0, 0, 0.5]), [ 0, 0, 1 ], 3, 'x')
mol_view.shape.add('text', list(8*rotaxes[1]+[0, 0, 0.5]), [ 1, 0, 0 ], 3, 'y')
mol_view.shape.add('text', list(8*rotaxes[2]+[0, 0.5, 0]), [ 0, 0.8, 0.2 ], 3, 'z')
mol_view.shape.add_arrow([0,0,0], list(8*rotaxes[0]), [ 0, 0, 1 ], 0.2, 'x')
mol_view.shape.add_arrow([0,0,0], list(8*rotaxes[1]), [ 1, 0, 0 ], 0.2, 'y')
mol_view.shape.add_arrow([0,0,0],  list(8*rotaxes[2]), [ 0, 0.8, 0.2 ], 0.2, 'z')



# text boxes
textgap=widgets.Text(value="{:.2f}".format(cell[2]),step=0.1,disabled=True,layout=textlayout)
textsurf=widgets.Text(value="{:.2f}".format(cell[0]*cell[1]),step=0.1,disabled=True,layout=textlayout)

textA=widgets.Text(value="{:.2f}".format(get_target_single(IR_ints,"A")),step=0.1,disabled=True,layout=targetlayout)
textR=widgets.Text(value="{:.2f}".format(get_target_single(R_ints,"R")),step=0.1,disabled=True,layout=targetlayout)
textP=widgets.Text(value="{:.2f}".format(get_target_single(prod_ints,"P")),step=0.1,disabled=True,layout=targetlayout)
textAav=widgets.Text(value="{:.2f}".format(get_target_single(IR_ints_av,"A")),step=0.1,disabled=True,layout=targetlayout)
textRav=widgets.Text(value="{:.2f}".format(get_target_single(R_ints_av,"R")),step=0.1,disabled=True,layout=targetlayout)
textPav=widgets.Text(value="{:.2f}".format(get_target_single(prod_ints_av,"P")),step=0.1,disabled=True,layout=targetlayout)



# create spectra
spectra_plot = widgets.Output()

with spectra_plot:
    fig=plt.figure(constrained_layout=True,num='Vibrational spectra')
    fig.set_size_inches(3.5, 3.) #(4.5, 3.6)
    ax2=fig.add_subplot(311)
    ax3=fig.add_subplot(312,sharex=ax2)
    ax1=fig.add_subplot(313,sharex=ax2) 
    
plt.rcParams.update({'font.size': 9})


pmin=0 
pmax=int((xmax-xmin)/res)
maxpr=1
maxI=1
maxR=1
maxpr0=0
maxI0=0
maxR0=0
if single_widget.value:
    maxpr0=np.max(prod_ints)
    maxI0=np.max(IR_ints)
    maxR0=np.max(R_ints)
    if broadening_widget.value=='broadened':
        ax3.fill_between(wn,R_spec,alpha=0.6,color='b',label='Raman')
        line1=ax3.plot(wn,R_spec,alpha=0.6,color='b',label='Raman')
        ax2.fill_between(wn,IR_spec,alpha=0.6,color='r',label='THz/IR') 
        line2=ax2.plot(wn,IR_spec,alpha=0.6,color='r',label='THz/IR')
        ax1.fill_between(wn,conv_spec,alpha=0.6,color='purple',label='Conversion') 
        line3=ax1.plot(wn,conv_spec,alpha=0.6,color='purple',label='Conversion')
        maxpr0=np.max(conv_spec[pmin:pmax]) 
        maxI0=np.max(IR_spec[pmin:pmax])
        maxR0=np.max(R_spec[pmin:pmax])
        c1 = mplcursors.cursor(line1)
        c2 = mplcursors.cursor(line2)
        c3 = mplcursors.cursor(line3)

    elif broadening_widget.value=='broadened+stick':
        ax3.fill_between(wn,(gammaR*math.pi)/2*R_spec,alpha=0.6,color='b',label='Raman')
        ax2.fill_between(wn,(gammaIR*math.pi)/2*IR_spec,alpha=0.6,color='r',label='THz/IR') 
        ax1.fill_between(wn,(gammaIR*math.pi)/2*(gammaR*math.pi)/2*conv_spec,alpha=0.6,color='purple',label='Conversion') 

        markerline, stemline, baseline, =ax2.stem(freqs,IR_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='THz/IR')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        c4 = mplcursors.cursor(markerline)
        markerline, stemline, baseline, =ax3.stem(freqs,R_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Raman')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        c5 = mplcursors.cursor(markerline)
        markerline, stemline, baseline, =ax1.stem(freqs,prod_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Conversion')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)
        c6 = mplcursors.cursor(markerline)

    elif broadening_widget.value=='stick':
        markerline, stemline, baseline, =ax2.stem(freqs,IR_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='THz/IR')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)
        c1 = mplcursors.cursor(markerline)
        markerline, stemline, baseline, =ax3.stem(freqs,R_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Raman')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        c2 = mplcursors.cursor(markerline)
        markerline, stemline, baseline, =ax1.stem(freqs,prod_ints,
                                                 'k',basefmt='k', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Conversion')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)
        c3 = mplcursors.cursor(markerline)
    maxpr=maxpr0
    maxI=maxI0
    maxR=maxR0
if av_widget.value:
    maxpr1=np.max(prod_ints_av) 
    maxI1=np.max(IR_ints_av) 
    maxR1=np.max(R_ints_av) 
    if broadening_widget.value=='broadened':
        ax3.fill_between(wn,R_spec_av,alpha=0.6,color='grey',label='Raman av')
        ax2.fill_between(wn,IR_spec_av,alpha=0.6,color='grey',label='THz/IR av')
        ax1.fill_between(wn,conv_spec_av,alpha=0.6,color='grey',label='Conv. av')
        maxpr1=np.max(conv_spec_av[pmin:pmax]) 
        maxI1=np.max(IR_spec_av[pmin:pmax])
        maxR1=np.max(R_spec_av[pmin:pmax])
           
    elif broadening_widget.value=='broadened+stick':
        ax3.fill_between(wn,(gammaR*math.pi)/2*R_spec_av,alpha=0.6,color='grey',label='Raman av')
        ax2.fill_between(wn,(gammaIR*math.pi)/2*IR_spec_av,alpha=0.6,color='grey',label='THz/IR av')
        ax1.fill_between(wn,(gammaIR*math.pi)/2*(gammaR*math.pi)/2*conv_spec_av,alpha=0.6,color='grey',label='Conv. av')

        markerline, stemline, baseline, =ax2.stem(freqs,IR_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='THz/IR')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
            
        markerline, stemline, baseline, =ax3.stem(freqs,R_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Raman')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        markerline, stemline, baseline, =ax1.stem(freqs,prod_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Conversion')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
            
    elif broadening_widget.value=='stick':
        markerline, stemline, baseline, =ax2.stem(freqs,IR_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='THz/IR')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        markerline, stemline, baseline, =ax3.stem(freqs,R_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Raman')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
        markerline, stemline, baseline, =ax1.stem(freqs,prod_ints_av,
                                                 'grey',basefmt='grey', markerfmt='ok',
                                                  use_line_collection=True,bottom=0,label='Conversion')
        plt.setp(stemline, linewidth = 1.25,alpha=0.6)
        plt.setp(markerline, markersize = 2,alpha=0.6)  
    
    maxpr=max(maxpr0, maxpr1) 
    maxI=max(maxI0,maxI1)
    maxR=max(maxR0,maxR1) 
plt.xlim(xmin,xmax)
ax1.set_ylim(-maxpr/100,1.2*maxpr)
ax2.set_ylim(-maxI/100,1.2*maxI)
ax3.set_ylim(-maxR/100,1.2*maxR)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax3.get_xticklabels(), visible=False)
ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
ax2.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
#  ax2.set_title(title)
plt.xlabel('Wavenumber /cm$^{-1}$')
ax2.set_ylabel('THz/IR',color='r')
ax3.set_ylabel('Raman',color='b')
ax1.set_ylabel('Conversion',color='purple')
ax1.yaxis.set_label_coords(-0.07,0.5)
ax2.yaxis.set_label_coords(-0.07,0.5)
ax3.yaxis.set_label_coords(-0.07,0.5)


# footnote on units
footnote=widgets.HTMLMath(
    value="""
    <br>
    <font size="-2">
    SMILES code: """
    + smiles + 
    """
    <br>
    Note on spectra units: 
    THz/IR [km\(\cdot\)mol\(^{-1}\)], 
    Raman [cm\(^{2}\cdot\)sr\(^{-1}\)], 
    Conversion [km\(\cdot\)mol\(^{-1}\cdot\)cm\(^{2}\cdot\)sr\(^{-1}\)]
    </font>
    """,
)


# connect callbacks
phi_widget.observe(on_rot_change, 'value')
theta_widget.observe(on_rot_change, 'value')
xi_widget.observe(on_rot_change, 'value')

laser_widget.observe(on_value_change, 'value')
T_widget.observe(on_value_change, 'value')
freq_widget.observe(on_value_change, 'value')
sclf_widget.observe(on_value_change, 'value')
broadening_widget.observe(on_value_change, 'value')
gammaIR_widget.observe(on_value_change, 'value')
gammaR_widget.observe(on_value_change, 'value')
av_widget.observe(on_value_change, 'value')
single_widget.observe(on_value_change, 'value')
rb_ir_beam.observe(on_value_change, 'value')
rb_rin_beam.observe(on_value_change, 'value')
rb_rout_beam.observe(on_value_change, 'value')



# display widgets
mol_title=widgets.Label(value="Molecular viewer",layout = widgets.Layout(display="flex", 
                                                justify_content="center",width='350px'))
gapsize=HBox([widgets.Label(value="Layer height /"+u'\u212B',layout=labellayout),textgap])
surface=HBox([widgets.Label(value="XY projection /"+u'\u212B\u00B2',layout=labellayout),textsurf])
anisotropy=HBox([widgets.Label(value="Anisotropy",layout=labellayout),widgets.Text(value="{:.2f}".format(aniso),step=0.1,
                                disabled=True,layout=textlayout)])

label1=widgets.Label(value=r'Chosen orientation',
                                layout = widgets.Layout(display="flex",justify_content="flex-end",width='150px'))
label2=widgets.Label(value=r'Orientation average',width='150px')

target_A=HBox([widgets.Label(value="A",layout=targetlabellayout),textA])
target_R=HBox([widgets.Label(value="R",layout=targetlabellayout),textR])
target_P=HBox([widgets.Label(value="P",layout=targetlabellayout),textP])

target_A_av=textAav
target_R_av=textRav
target_P_av=textPav
        
targets1=VBox([label1,target_A,target_R,target_P])#,layout=Layout(justify_content='space-between')))
targets2=VBox([label2,target_A_av,target_R_av,target_P_av])#,layout=Layout(justify_content='space-between'))) 
targets=HBox([targets1,targets2],layout=Layout(justify_content='space-around')) 


polarization  = [
    widgets.VBox([laser_widget,T_widget]),
    widgets.VBox([freq_label,freq_widget,sclf_widget,broadening_widget,gammaIR_widget,gammaR_widget,single_widget,av_widget]),
    widgets.VBox([rb_ir_beam,rb_rin_beam,rb_rout_beam]),
    widgets.VBox([phi_widget,theta_widget,xi_widget])
                ]

controls = widgets.Accordion(children=polarization,layout=widgets.Layout(width='360px'),selected_index=3)
controls.set_title(0, 'Set experimental parameters')
controls.set_title(1, 'Set plotting parameters')
controls.set_title(2, 'Set field polarization')
controls.set_title(3, 'Set molecule orientation')

controls.layout.height = '500px'
mol_panel=VBox([mol_title,mol_view,gapsize,surface,anisotropy])
spectrum_panel=VBox([spectra_plot,targets])
mol_panel.layout.width = '300px'
main_panel=HBox([controls,mol_panel,spectrum_panel], layout=Layout(justify_content='space-around'))
display(VBox([main_panel,footnote]))

### Similar molecules

In [8]:
# Similarity to other molecules
def read_smiles_data(fname):
    SAMsmiles=[]
    scode=[]
    with open(fname) as inpfile:
        line = inpfile.readline()
        line = inpfile.readline()
        while line:
            scode.append(line.split()[0])
            SAMsmiles.append(line.split()[1])
            line = inpfile.readline()
    return SAMsmiles,scode

def show_similar(smfile):
    SAMsmiles,scode=read_smiles_data(smfile)
    ms=[]
    rad=1
    for s in SAMsmiles:
        if Chem.MolFromSmiles(s)==None:
            ms.append(None)
            continue
        m=Chem.MolFromSmiles(Chem.MolToSmiles(Chem.MolFromSmiles(s)))
        ms.append(m) 
    fps=[]
    for x in ms:
        if x==None:
            fps.append(None)
            continue
        fps.append(AllChem.GetMorganFingerprint(x, radius=rad, useCounts=1,useFeatures=0))

    molfp = AllChem.GetMorganFingerprint(mol, radius=rad, useCounts=1,useFeatures=0)
    sim=np.zeros((len(fps)))
    for j, fps2 in enumerate(fps):
            if fps2==None:
                sim[j]=0
                continue
            sim[j]=DataStructs.DiceSimilarity(molfp,fps2)

    mmol=np.argsort(sim)
    simil=[]
    for mm in mmol:
        simil.append(DataStructs.DiceSimilarity(molfp,fps[mm]))

    
    return scode,mmol,simil,SAMsmiles

In [14]:
from IPython.core.display import HTML
import pandas as pd

def path_to_image_html(path):
    return '<img src="'+ path + '" width="200" >'

def show_table(page,SAM,SH,fname,mmol,simil,allsmiles):
    nummols=10
    nmols=len(mmol)
    minmol=max(0,nmols-(page)*nummols)
    maxmol=nmols-(page-1)*nummols
    molnums=np.arange(nmols-maxmol+1,nmols-minmol+1)
    pd.options.display.float_format = '{:.2f}'.format

    bestmols=reversed(mmol[minmol:maxmol])
    images = []
    mollinks=[]
    modelinks=[]
    msmiles=[]
    codes=[]
    for m in pd.Series(bestmols,dtype='int'):
        molcode=(fname[m]).split("-",1)[1]
        codes.append(molcode)
        msmiles.append(Chem.MolToSmiles(Chem.MolFromSmiles(allsmiles[m])))
        if SAM:
            images.append("data_SAM/{}.png".format(molcode))
        elif SH:
            images.append("data_SH/{}.png".format(molcode))
        else:
            images.append("data_SAu/{}.png".format(molcode))
        molurl="<a href="+ url+ "/molecule.ipynb?Molcode="+molcode+"&SH={}".format(SH)+">Go to molecule page</a>"
        modeurl="<a href="+url+"/modes.ipynb" +"?Molcode="+molcode+"&SH={}".format(SH)+">Check normal modes</a>"
        mollinks.append(molurl)
        modelinks.append(modeurl)

    if SAM:
        df = pd.DataFrame({
        'Index' : pd.Series(molnums,dtype='int'),
        #'Code': pd.Series(codes),
        'Molecule': pd.Series(images),
        'Similarity' : pd.Series(reversed(simil[minmol:maxmol]),dtype='f4'),
        'SMILES': pd.Series(msmiles),
        })
    
    else:
        df = pd.DataFrame({
        'Index' : pd.Series(molnums,dtype='int'),
        #'Code': pd.Series(codes),
        'Molecule': pd.Series(images),
        'Similarity' : pd.Series(reversed(simil[minmol:maxmol]),dtype='f4'),
        'Link to molecule page': pd.Series(mollinks),
        'Link to normal mode page': pd.Series(modelinks),
        'SMILES': pd.Series(msmiles),
        })

    
    s = df.style.format(formatter=dict(Molecule=path_to_image_html,Similarity="{:.2f}"))
    cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
    }
    index_names = {
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
    }
    headers = {
        'selector': 'th:not(.index_name)',
        'props': 'background-color: grey; color: white;'
    }
    s.set_table_styles([cell_hover, index_names, headers])

    s.set_properties(**{'background-color': 'white',
                           'color': 'black',
                           'border-color': 'black',
                        'border-style':'solid',
                        'border-width':'1.5px',
                       })
    display(HTML(s.hide_index().render()))

In [15]:
if SH:
    fname,mmol,simil,allsmiles=show_similar(smfile="data_SH/frequencies_SH.txt") 
else:
    fname,mmol,simil,allsmiles=show_similar(smfile="data_SAu/frequencies.txt")

nummols=10
pages=np.arange(1,round(len(mmol)/nummols+1))

page1=widgets.Dropdown(options=pages,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)

out1=widgets.interactive_output(show_table,{'page':page1,'SAM':fixed(0),'SH':fixed(SH),
                                           'fname':fixed(fname),'mmol':fixed(mmol),
                                            'simil':fixed(simil),'allsmiles':fixed(allsmiles)})

fname2,mmol2,simil2,allsmiles2=show_similar(smfile="data_SAM/smiles_SAM_materials.txt",
                                       )
pages2=np.arange(1,round(len(mmol2)/nummols+1))

page2=widgets.Dropdown(options=pages2,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)
out2=widgets.interactive_output(show_table,{'page':page2,'SAM':fixed(1),'SH':fixed(SH),
                                            'fname':fixed(fname2),'mmol':fixed(mmol2),
                                            'simil':fixed(simil2),'allsmiles':fixed(allsmiles2)})

similar  = [
VBox([page1,out1]),
VBox([page2,out2])
                ]

controls = widgets.Accordion(children=similar,selected_index=None)
controls.set_title(0, 'Similar molecules in the database')
controls.set_title(1, 'Similar self-assembly materials')

display(controls)

Accordion(children=(VBox(children=(Dropdown(description='Page:', layout=Layout(width='150px'), options=(1, 2, …